In [1]:
from config import config

In [2]:
%load_ext autoreload
%autoreload 2
from model_utils.thswad import load_model_from_transe
from model_utils.trainer import THSWADTrainer
from utils.ds import *

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [3]:
import pickle
import json
from torch.utils.data import DataLoader

BS = 256
VAL_BS = 4
PREV_CNT = 5


def make_item2id(el_single_train):
    ds_ = DialogDataset(el_single_train['entity_ids'], 'pad', PREV_CNT)
    items2id = {}
    for i, (prev, next_id) in enumerate(ds_):
        for item in prev:
            if item not in items2id and item != 'pad':
                items2id[item] = len(items2id)
        if next_id not in items2id and next_id != 'pad':
            items2id[next_id] = len(items2id)
    return items2id

with open(config['dataset']['postprocess']['train'], 'r') as f:
    el_single_results = json.load(f)
items2id = make_item2id(el_single_results)
id2items = list(items2id.keys())


def make_dataloader(el_single_res, wikidata_ds, items2id=items2id, batch_size=BS, shuffle=True):
    ds = DialogDataset(el_single_res['entity_ids'], len(items2id), PREV_CNT, items2id)
    print(len(ds))
    dl = DataLoader(ds, batch_size=batch_size, shuffle=shuffle)
    wdl = DataLoader(wikidata_ds, batch_size=batch_size, shuffle=shuffle)
    return TSHWADLoader(wdl, dl)


with open(config['dataset']['postprocess']['val'], 'r') as f:
    el_val = json.load(f)
val_loader = make_dataloader(el_val, [], batch_size=VAL_BS, shuffle=False)
train_loader = make_dataloader(el_single_results, [], batch_size=VAL_BS, shuffle=False)


/data/home/admin/dream-entity-recommendation/utils/ds.py:64: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if np.all(np.array(prev) == pad_item):


24610
329191


In [4]:
from collections import defaultdict
t = 0
items_same = defaultdict(lambda: 0) 
items = defaultdict(lambda: 0)
all_items_set = set()
for i, b_ in enumerate(train_loader):
    s = 0.
    for x, y in zip(b_['ratings'][0], b_['ratings'][1]):
        all_items_set.add(y.item())
        for v in x:
            all_items_set.add(v.item())

        s += y in x
        items[y.item()] += 1
        if y in x:
            items_same[y.item()] += 1
    t += s / b_['ratings'][0].shape[0]

t / i, len(all_items_set)

(0.1913172209598568, 18894)

In [5]:
with open(config['wikidata']['transe'], "rb") as f:
    transe = pickle.load(f)

In [6]:
item_id2entity_id = []
for id, item in enumerate(id2items):
    item_id2entity_id.append(transe.graph.entity2id[item])

In [7]:
item_embeddings = np.zeros((len(item_id2entity_id) + 1, transe.solver.entity_embeddings.shape[1]))
for i, ent in enumerate(item_id2entity_id):
    item_embeddings[i] = transe.solver.entity_embeddings[ent]

In [8]:
def get_topk_numpy(a, v, k=10):
    x = np.sum(np.abs(a - v), 1)
    t = np.argpartition(x, k)[:k]
    kv = [(x[v], v) for v in t]
    kv.sort(key=lambda v: v[0])
    return [v[1] for v in kv]


def get_top_k_entities_numpy(entity, k=10):
    return get_topk_numpy(item_embeddings, item_embeddings[entity])

In [11]:
from tqdm import tqdm
map10 = 0.
map5 = 0.
accuracy = 0.
for i, b_ in tqdm(enumerate(val_loader)):
    map10_b = 0.
    map5_b = 0.
    accuracy_b = 0.
    for x, y in zip(b_['ratings'][0], b_['ratings'][1]):
        le = x[-1]
        y = y
        top10 = get_top_k_entities_numpy(le)
        map10_b += y in top10
        map5_b += y in top10[:5]
        accuracy_b += y == top10[0]

    map10 += map10_b / b_['ratings'][0].shape[0]
    map5 += map5_b / b_['ratings'][0].shape[0]
    accuracy += accuracy_b / b_['ratings'][0].shape[0]

map10 / i, map5 / i, accuracy / i

6153it [37:48,  2.71it/s]


(0.06424739921976592, 0.0588832899869961, tensor(0.0493))

In [12]:
print('map@10 %.3f' % (map10 / i), 'map@5 %.3f' % (map5 / i), 'accuracy %.3f' % (accuracy / i), sep=', ')

map@10 0.064, map@5 0.059, accuracy 0.049


Daily dialog map@10 0.097, map@5 0.093, accuracy 0.087

Dream vs ChatGPT map@10 0.155, map@5 0.141, accuracy 0.123

Topical-Chat map@10 0.064, map@5 0.059, accuracy 0.049